In [71]:
import pandas as pd 
import geopandas as gpd 
from shapely.geometry import Point
import numpy as np 

In [58]:
# Load Wilayas Features and calculate polygon centroids
wilayas_gjson = '../dz-admin/wilayas_48.geojson'
wilayas_gdf = gpd.read_file(wilayas_gjson)
wilayas_gdf['centroid'] = wilayas_gdf['geometry'].centroid
wilayas_gdf.head()

<ipython-input-58-6fe9d1a6e253>:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  wilayas_gdf['centroid'] = wilayas_gdf['geometry'].centroid


,code,nom,nom_maj,nom_ar,geometry,centroid
0,1,Adrar,ADRAR,أدرار,"MULTIPOLYGON (((1.33461 20.72282, 1.32667 20.7...",POINT (-0.62174 26.01400)
1,2,Chlef,CHLEF,الشلف,"MULTIPOLYGON (((1.55326 36.02990, 1.55325 36.0...",POINT (1.25669 36.23826)
2,3,Laghouat,LAGHOUAT,اﻷغواط,"MULTIPOLYGON (((2.88464 32.88137, 2.87848 32.8...",POINT (2.72955 33.60757)
3,4,Oum El Bouaghi,OUM EL BOUAGHI,أم البواقي,"MULTIPOLYGON (((7.47320 35.52005, 7.47227 35.5...",POINT (7.06527 35.83646)
4,5,Batna,BATNA,باتنة,"MULTIPOLYGON (((6.10102 35.05459, 6.10090 35.0...",POINT (5.87455 35.41969)


In [72]:
wilayas_gdf = wilayas_gdf.set_crs("EPSG:4326")
wilayas_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [73]:
# Load Tagreted dataset and set to_check to False assuming all geocoded coordinates are OK 
pois = pd.read_csv('../dz-datasets/accountants/personnes_physiques.csv')
pois['to_check'] = False
pois.head()

,num,num_inscription,nom,prenom,adresse,wilaya,tel,fax,email,lat,lon,to_check
0,1,1707,ABADA,MOHAMMED CHERIF,"58,RUE ABDELHAMID BENBADIS - ANNABA",Annaba,38862288,NaN,abadamedcherif@gmail.com,36.914208,7.742667,False
1,2,202,ABADA eps GUERFI,FARIDA,CITE PLEINE OUEST 687 LOGTS BLOC 13 - ANNABA,Annaba,38542953,38761462,NaN,36.900380,7.734217,False
2,3,1708,ABADOU,MOHAMMED,CITE 556 LOGTS Bt 110 N'111FERDJIOUA - MILA,Mila,NaN,NaN,NaN,NaN,NaN,False
3,4,214,ABAR,MOHAMEO,"32, RUE DES FUSILLES,EL ANNASS ER -ALGER",Alger,NaN,21684890,cab_abar@yahoo.fr,NaN,NaN,False
4,5,1073,ABBACI,MOHAMED,BO EL MOKRANI - CHLEF,Chlef,27775003,NaN,NaN,36.157966,1.337282,False


In [74]:
# Add wilayas codes 
wilayas_codes = wilayas_gdf[['code', 'nom']]
wilayas_codes = wilayas_codes.rename(columns={'nom': 'nom_wil'})
pois_wilayas = pois.merge(wilayas_codes, left_on='wilaya', right_on='nom_wil')
pois_wilayas.drop(columns=['nom_wil'], inplace=True)
pois_wilayas.head()

,num,num_inscription,nom,prenom,adresse,wilaya,tel,fax,email,lat,lon,to_check,code
0,1,1707,ABADA,MOHAMMED CHERIF,"58,RUE ABDELHAMID BENBADIS - ANNABA",Annaba,38862288,NaN,abadamedcherif@gmail.com,36.914208,7.742667,False,23
1,2,202,ABADA eps GUERFI,FARIDA,CITE PLEINE OUEST 687 LOGTS BLOC 13 - ANNABA,Annaba,38542953,38761462,NaN,36.900380,7.734217,False,23
2,26,35,ABED,BOUALEM,"RUE DlDOUCHE MOURAD, EL HADJAR - ANNABA",Annaba,38892681,NaN,becblm23@gmail.com,36.807471,7.738913,False,23
3,73,205,AISSAOUI,NASREDDINE,AVENUE DE L'AVANT PORT - ANNABA,Annaba,NaN,NaN,aissaoulcompta2003@yahoo.fr,36.899954,7.765460,False,23
4,124,366,ALLOUT,MOHAMMED,"20, RUE LAMARA ABDELKADER - ANNABA",Annaba,38864148,38864148,cecfallout@hotmall.fr,36.898577,7.759510,False,23


In [75]:
# Convert pandas DataFrame to GeoPandas 
pois_wilayas['geometry'] = pois_wilayas.apply(lambda row: Point(row.lon, row.lat), axis=1)
pois_gdf  = gpd.GeoDataFrame(pois_wilayas)
pois_gdf.head()

,num,num_inscription,nom,prenom,adresse,wilaya,tel,fax,email,lat,lon,to_check,code,geometry
0,1,1707,ABADA,MOHAMMED CHERIF,"58,RUE ABDELHAMID BENBADIS - ANNABA",Annaba,38862288,NaN,abadamedcherif@gmail.com,36.914208,7.742667,False,23,POINT (7.74267 36.91421)
1,2,202,ABADA eps GUERFI,FARIDA,CITE PLEINE OUEST 687 LOGTS BLOC 13 - ANNABA,Annaba,38542953,38761462,NaN,36.900380,7.734217,False,23,POINT (7.73422 36.90038)
2,26,35,ABED,BOUALEM,"RUE DlDOUCHE MOURAD, EL HADJAR - ANNABA",Annaba,38892681,NaN,becblm23@gmail.com,36.807471,7.738913,False,23,POINT (7.73891 36.80747)
3,73,205,AISSAOUI,NASREDDINE,AVENUE DE L'AVANT PORT - ANNABA,Annaba,NaN,NaN,aissaoulcompta2003@yahoo.fr,36.899954,7.765460,False,23,POINT (7.76546 36.89995)
4,124,366,ALLOUT,MOHAMMED,"20, RUE LAMARA ABDELKADER - ANNABA",Annaba,38864148,38864148,cecfallout@hotmall.fr,36.898577,7.759510,False,23,POINT (7.75951 36.89858)


In [76]:
# Set the Coordinate reference system
pois_gdf = pois_gdf.set_crs("EPSG:4326")
pois_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [82]:
# Check for rows with no coordiantes
no_coords_gdf = pois_gdf[(pois_gdf['lat'].isnull() | pois_gdf['lat'].isnull())]

# Merge with Wialyas DataFrame and set geometry to centroids
no_coords_gdf = no_coords_gdf.merge(wilayas_gdf, left_on='code', right_on='code')
no_coords_gdf['geometry'] = no_coords_gdf['centroid']
no_coords_gdf['to_check'] = True

with_coords_gdf = no_coords_gdf.drop(columns=['geometry_x'], axis=1)

no_coords_gdf 

,num,num_inscription,nom_x,prenom,adresse,wilaya,tel,fax,email,lat,lon,to_check,code,geometry_x,nom_y,nom_maj,nom_ar,geometry_y,centroid,geometry
0,1798,148,REDJATI,SEDDIK,CITE 08 MARS VILLA N' 3 COOP EL ALOUANE - ANNABA,Annaba,38541563,NaN,NaN,NaN,NaN,True,23,POINT (nan nan),Annaba,ANNABA,عنابة,"MULTIPOLYGON (((7.48643 36.67331, 7.48571 36.6...",POINT (7.54325 36.83843),POINT (7.54325 36.83843)
1,3,1708,ABADOU,MOHAMMED,CITE 556 LOGTS Bt 110 N'111FERDJIOUA - MILA,Mila,NaN,NaN,NaN,NaN,NaN,True,43,POINT (nan nan),Mila,MILA,ميلة,"MULTIPOLYGON (((6.01335 35.89529, 6.01206 35.8...",POINT (6.14584 36.29332),POINT (6.14584 36.29332)
2,23,446,ABDESSEMED,ABDELHAFID,CITE 20 AOUT Bt Bl N'03 CHELGHOUM EL AID - MILA,Mila,31457424,31457424,asamed.af@gmail.com,NaN,NaN,True,43,POINT (nan nan),Mila,MILA,ميلة,"MULTIPOLYGON (((6.01335 35.89529, 6.01206 35.8...",POINT (6.14584 36.29332),POINT (6.14584 36.29332)
3,1024,1938,FERRADJI,ALDJIA,"CITE 18 LOGTS B1 01 ""CHÂTEAU D'EAU"" - MILA",Mila,31475302,NaN,leuars@yahoo.com,NaN,NaN,True,43,POINT (nan nan),Mila,MILA,ميلة,"MULTIPOLYGON (((6.01335 35.89529, 6.01206 35.8...",POINT (6.14584 36.29332),POINT (6.14584 36.29332)
4,4,214,ABAR,MOHAMEO,"32, RUE DES FUSILLES,EL ANNASS ER -ALGER",Alger,NaN,21684890,cab_abar@yahoo.fr,NaN,NaN,True,16,POINT (nan nan),Alger,ALGER,الجزائر,"MULTIPOLYGON (((2.95544 36.61091, 2.95366 36.6...",POINT (3.05748 36.70539),POINT (3.05748 36.70539)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,133,447,AMAR ROUANA,LIAZID,CITE ER NOVEMBRE 54 BT 31 N°lO -SKIKDA,Skikda,NaN,NaN,NaN,NaN,NaN,True,21,POINT (nan nan),Skikda,SKIKDA,سكيكدة,"MULTIPOLYGON (((6.88006 36.56313, 6.87913 36.5...",POINT (6.83331 36.78202),POINT (6.83331 36.78202)
63,2016,2058,TOBBA,FATIMA ZOHRA,CITE GUEHDOUR TAHAR Bt N' 03 Appart N' 05 -G...,Guelma,37117550,37117550,NaN,NaN,NaN,True,24,POINT (nan nan),Guelma,GUELMA,قالمة,"MULTIPOLYGON (((7.04176 36.20559, 7.04187 36.2...",POINT (7.39568 36.39577),POINT (7.39568 36.39577)
64,333,1869,BELMAMOUNE,BOUZIANE,RUE AVOCAT ABED N' 21- EL BAYADH,El Bayadh,NaN,NaN,NaN,NaN,NaN,True,32,POINT (nan nan),El Bayadh,EL BAYADH,البيض,"MULTIPOLYGON (((-0.02094 32.71034, -0.02327 32...",POINT (0.91159 32.56540),POINT (0.91159 32.56540)
65,348,1297,BENABDELMOUMENE,MOHAMMED,ROUTE D'AFLOU - EL BAYADH,El Bayadh,NaN,NaN,NaN,NaN,NaN,True,32,POINT (nan nan),El Bayadh,EL BAYADH,البيض,"MULTIPOLYGON (((-0.02094 32.71034, -0.02327 32...",POINT (0.91159 32.56540),POINT (0.91159 32.56540)


In [85]:
# Update the Targeted DataFrame by adding Wilayas geometries and centroids 
pois_gdf = pois_gdf[(pois_gdf['lat'].notnull() & pois_gdf['lat'].notnull())]
pois_gdf = pois_gdf.merge(wilayas_gdf, left_on='code', right_on='code')
pois_gdf.rename(columns={"geometry_x": "geometry"}, inplace=True)

# Concat the Targeted DataFrame with rows without coordinates and approximated with Wilayas centroids 
pois_gdf = pd.concat([pois_gdf, with_coords_gdf])
pois_gdf[(pois_gdf['lat'].isnull() | pois_gdf['lon'].isnull())]

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [86]:
# Sort the DataFrame by code (Wilaya number) and reset the index
pois_gdf = pois_gdf.sort_values(by=['code'],ignore_index=True)
pois_gdf.reset_index(drop=True)
pois_gdf.head()

,num,num_inscription,nom_x,prenom,adresse,wilaya,tel,fax,email,lat,...,nom_y,nom_maj_x,nom_ar_x,geometry_y,centroid_x,nom,nom_maj_y,nom_ar_y,geometry_y,centroid_y
0,61,1632,AHRES,HASSEN,RUE KADDOUR BELLITIM - ADRAR,Adrar,49964925,NaN,hassanahres@gmail.com,27.882773,...,Adrar,ADRAR,أدرار,"MULTIPOLYGON (((1.33461 20.72282, 1.32667 20.7...",POINT (-0.62174 26.01400),Adrar,ADRAR,أدرار,"MULTIPOLYGON (((1.33461 20.72282, 1.32667 20.7...",POINT (-0.62174 26.01400)
1,1865,2038,SAIDI,ABDERRAFIR,RUE ABDELKA DER BENSLIMANE EN FACE PARC DTP - ...,Adrar,NaN,NaN,saidiabderaflr@gmail.com,33.618970,...,Adrar,ADRAR,أدرار,"MULTIPOLYGON (((1.33461 20.72282, 1.32667 20.7...",POINT (-0.62174 26.01400),Adrar,ADRAR,أدرار,"MULTIPOLYGON (((1.33461 20.72282, 1.32667 20.7...",POINT (-0.62174 26.01400)
2,1517,1189,MALKI,RACHID,RUE BELGHIT MOHAMMED N°20 - ADRAR,Adrar,49967999,NaN,malki2004@yahoo.fr,28.017440,...,Adrar,ADRAR,أدرار,"MULTIPOLYGON (((1.33461 20.72282, 1.32667 20.7...",POINT (-0.62174 26.01400),Adrar,ADRAR,أدرار,"MULTIPOLYGON (((1.33461 20.72282, 1.32667 20.7...",POINT (-0.62174 26.01400)
3,1418,1355,LAAFOU,AHMED,CITE 175 LOGTS ROUTETILILANE - ADRA R,Adrar,0499667OO,NaN,NaN,27.898056,...,Adrar,ADRAR,أدرار,"MULTIPOLYGON (((1.33461 20.72282, 1.32667 20.7...",POINT (-0.62174 26.01400),Adrar,ADRAR,أدرار,"MULTIPOLYGON (((1.33461 20.72282, 1.32667 20.7...",POINT (-0.62174 26.01400)
4,327,1213,BELLAOUI,ABDERRAHMANE,CITE 05 JUILLET 1962 - ADRAR,Adrar,NaN,49951555,NaN,28.017440,...,Adrar,ADRAR,أدرار,"MULTIPOLYGON (((1.33461 20.72282, 1.32667 20.7...",POINT (-0.62174 26.01400),Adrar,ADRAR,أدرار,"MULTIPOLYGON (((1.33461 20.72282, 1.32667 20.7...",POINT (-0.62174 26.01400)


In [87]:
# Check if coordinates are within Wilayas polygons
pois_gdf['coords_in_wil'] = [pois_gdf.geometry[i].within(pois_gdf.geometry_y[i]) for i in range(pois_gdf.geometry.count())] 
pois_gdf[pois_gdf.coords_in_wil == False]

KeyError: 0

In [525]:
# Update to_check flag to take into account points outside Wilays and without geocoding coordinates
#final_gdf = angem_gdf
#final_gdf['to_check'] = (angem_gdf['to_check'] == True) | (angem_gdf['coords_in_wil'] == False)   

# Remove unecessary columns
#final_gdf = final_gdf.drop(['geometry_y', 'code_wil', 'centroid', 'WILAYA', 'coords_in_wil'], axis=1)

#final_gdf[final_gdf.to_check == True]

,code,nom,type,adresses,e-mail,fax,tel,wilayas de rattachement,lat,lon,to_check,geometry
4,5,BATNA,Agence,CITE NACER CITE ADMINISTRATIVE BLOC 02 BATNA,batna@angem.dz,033 25 39 79,033 25 39 78,BATNA,NaN,NaN,True,POINT (5.87455 35.41969)
7,7,BISKRA,Antenne,BLOC N°06 CITE 200/500(PRES DE LA MOSQUEE EL F...,arbiskra@angem.dz,033 51 53 17,033 51 53 17,"Biskra, El Oued, Khenchla, Tebessa",37.172745,10.033169,True,POINT (10.03317 37.17275)
8,7,BISKRA,Agence,BLOC N° 06 CITE 200/500 (PRES DE LA MOSQUEE EL...,biskra@angem.dz,033 51 53 55,033 51 53 56,BISKRA,33.140374,5.595066,True,POINT (5.59507 33.14037)
12,10,BOUIRA,Agence,CITE 56 LGTS BT F ENTREE 02 A COTE DU STADE MU...,bouira@angem.dz,026 72 02 72,026 72 02 72,BOUIRA,40.520423,22.971676,True,POINT (22.97168 40.52042)
25,20,SAIDA,Agence,BOULVARD FRANZ FANON EX MOHAFADAT DE SAIDA,saida@angem.dz,048 41 02 53,048 41 00 94,SAIDA,NaN,NaN,True,POINT (0.28119 34.73743)
37,29,MASCARA,Agence,RUE CHEMMOUM MOKHTAR MASCARA,mascara@angem.dz,045 71 89 98,045 71 89 36,MASCARA,NaN,NaN,True,POINT (0.15926 35.39167)
42,33,ILLIZI,Agence,PLACE EL MAGHREB EL ARABI ILLIZI,illizi@angem.dz,029 41 16 10,029 41 16 10,ILLIZI,35.856345,-0.312907,True,POINT (-0.31291 35.85634)
50,41,SOUK AHRAS,Agence,CITE ADMINISTRATIVE 80 BUREAUX EL BORDJ EL KAB...,soukahras@angem.dz,037 72 43 86,037 72 43 87,SOUK AHRAS,35.515206,0.300470,True,POINT (0.30047 35.51521)
53,43,MILA,Agence,CITE BEN GUERBA FACE APW MILA,mila@angem.dz,031 47 48 16,031 47 47 44,MILA,47.255310,-122.472227,True,POINT (-122.47223 47.25531)
55,45,NAAMA,Agence,CITE ADMINISTRATIVE 50 BUREAUX BT 146 NAAMA,naama@angem.dz,049 59 50 09,049 59 56 63,NAAMA,NaN,NaN,True,POINT (-0.76827 33.30472)


In [528]:
# Write to a GeoPackage file

#gpd.GeoDataFrame(final_gdf).to_file("angem_to_check.geojson", driver='GeoJSON')